#I. Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%env HOME=/content/drive/MyDrive
%cd ~/Research/ner/baidu_model/
!ls -lrt

env: HOME=/content/drive/MyDrive
/content/drive/MyDrive/Research/ner/baidu_model
total 792742
-rw------- 1 root root 407141549 Aug 22 00:50 MT1_2.zip
-rw------- 1 root root    325021 Aug 24 03:01 test.txt
-rw------- 1 root root 404284553 Aug 31 20:41 final_intent_MT1.zip
drwx------ 2 root root      4096 Aug 31 23:15 MT1_2
drwx------ 2 root root      4096 Aug 31 23:15 non_0_MT1_6
-rw------- 1 root root         0 Sep  9 01:28 prediction_set_test.txt
-rw------- 1 root root      3478 Sep  9 01:30 Baidu_eval_generation_wTemplate_IntentModel.py
drwx------ 2 root root      4096 Sep  9 21:26 pred_generation


#II. Get model, label and log validation template, etc.

In [ ]:
import json

f1 = open('/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_logs.json','r')
log = json.load(f1)
f1.close()

In [ ]:
f2 = open('/content/drive/MyDrive/Research/ner/intent_classifier/final_result/updated_DSTC10_clean_val_labels.json','r')
label = json.load(f2)
f2.close()

In [ ]:
f3 = open('/content/drive/MyDrive/Research/ner/intent_classifier/final_result/kb_eval_final.json','r')
kb = json.load(f3)
f3.close()

In [ ]:
import pickle
f4 = open('/content/drive/MyDrive/Research/ner/text_classifier/final_results/finale.pkl','rb')
cluster = pickle.load(f4)
f4.close()

In [ ]:
print(cluster,len(cluster),sep='\n')

{1: {'restaurant_120325_14', 'restaurant_120331_11', 'restaurant_120106_14', 'restaurant_120085_12', 'attraction_100057_3', 'restaurant_120305_15', 'restaurant_120308_13', 'restaurant_120159_14', 'restaurant_120095_9', 'restaurant_120165_14', 'restaurant_120353_17', 'attraction_100171_1', 'restaurant_120472_13', 'attraction_100045_2', 'attraction_100043_1', 'restaurant_120417_15', 'restaurant_120324_15', 'restaurant_120315_12', 'restaurant_120269_14', 'restaurant_120333_17', 'attraction_100084_0', 'restaurant_120463_15', 'restaurant_120004_13', 'restaurant_120058_15', 'hotel_27_14', 'restaurant_120030_17', 'restaurant_120450_14', 'restaurant_120102_17', 'restaurant_120046_8', 'restaurant_120515_16', 'hotel_110111_9', 'restaurant_120087_15', 'hotel_23_4', 'restaurant_120167_18', 'restaurant_120214_12', 'restaurant_120037_14', 'restaurant_120475_0', 'restaurant_120072_13', 'attraction_100138_4', 'restaurant_120314_19', 'hotel_15_9', 'restaurant_120144_2', 'restaurant_120006_9', 'restaura

In [ ]:
 #Reference from baidu model data generation
 import argparse
 import json
 import string
 from tqdm import tqdm




knowledge = kb

def __convert_to_tgt(domain, entity, doc):
    field_names = ["[DOMAIN]", "[ENTITY]", "[TITLE]", "[BODY]"]
    field_values = [domain]
    if entity["name"] is not None:
        field_values.append(entity["name"])
    else:
        field_values.append("*")
    field_values.append(" ".join(doc["title"].split()))
    field_values.append(" ".join(doc["body"].split()))
    if args.do_lowercase:
        field_values = [x.lower() for x in field_values]

    fields = [name + " " + value for name, value in zip(field_names, field_values)]
    return " ".join(fields)

with open("test.txt", "w") as out_f:
    out_f.write("src\ttgt\n")
    preds = label
    for log, pred in zip(log, tqdm(preds, desc="Generate task2 input dataset")):
        if not pred["target"]:
            # Only select one domain.
            continue
        dialog = []
        for turn in log:
            turn["text"] = " ".join(turn["text"].split())
            dialog.append(f"{turn['text']}\x01{1 if turn['speaker'] == 'U' else 0}")
        dialog = " [SEP] ".join(dialog)

        for domain in knowledge:
            for entity_id in knowledge[domain]:
                entity = knowledge[domain][entity_id]
                for doc_id in entity["docs"]:
                    doc = entity["docs"][doc_id]
                    tgt = __convert_to_tgt(domain, entity, doc)
                    out_f.write(f"{dialog}\t{tgt}\1{0}\n")


## Initialize Model

In [ ]:
%cd non_0_MT1_6/
!ls -lrt

/content/drive/My Drive/Research/ner/baidu_model/6
total 428107
-rw------- 1 root root 438374583 Aug 31 19:10 pytorch_model.bin
-rw------- 1 root root      6205 Aug 31 19:10 config.json


In [ ]:
import torch

model_name = "bert-base-uncased"
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install transformers

In [ ]:
#Loading Tokenizer, Config, and Model from drive
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForSequenceClassification

model_name = "bert-base-uncased"

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, padding = True, truncation = True)
model = AutoModelForSequenceClassification.from_pretrained(".")

# Set device
model.to(device)

In [ ]:
#Checks for dict weight
state_dict = model.state_dict()
state_dict["bert.embeddings.word_embeddings.weight"]

tensor([[ 0.0295, -0.1416, -0.0489,  ..., -0.0529, -0.0170, -0.1349],
        [ 0.0222, -0.1429, -0.0394,  ..., -0.0433, -0.0214, -0.1196],
        [ 0.0242, -0.1352, -0.0515,  ..., -0.0459, -0.0236, -0.1178],
        ...,
        [ 0.0136, -0.1350, -0.0399,  ..., -0.0442, -0.0181, -0.1248],
        [ 0.0136, -0.1455, -0.0275,  ..., -0.0303, -0.0173, -0.1330],
        [ 0.0233, -0.1281, -0.0648,  ..., -0.0512, -0.0470, -0.1177]])

In [ ]:
%cd ../

/content/drive/My Drive/Research/ner/baidu_model


#III. Prepare validation Data

In [ ]:
n = 0
for i in range(len(log)):
    if label[i]['target'] == 1:
        n+=1
print(n)

104


In [ ]:
#R
#Extra Method for immediate testing or use
import string
from tqdm import tqdm

def _get_title_body_pair(kb, doc_id):
    #gets title_ex and body_ex
    cat,id,d_id = doc_id.split('_')
    body_ex = kb[cat][id]['docs'][d_id]['body_ex']
    title_ex = kb[cat][id]['docs'][d_id]['title_ex']
    return (title_ex, body_ex)

def __convert_to_tgt(domain, entity, pair):
    field_names = ["[DOMAIN]", "[ENTITY]", "[TITLE]", "[BODY]"]
    field_values = [domain]
    if entity is not None:
        field_values.append(entity)
    else:
        field_values.append("*")
    field_values.append(" ".join(pair[0].split()))
    field_values.append(" ".join(pair[1].split()))

    fields = [name + " " + value for name, value in zip(field_names, field_values)]
    return " ".join(fields)

In [ ]:
#R
def single_test(title):
    result = tokenizer(title, return_tensors="pt")
    result.to(device)
    model.eval()
    outputs = model(**result)
    #print("Output Dictionary Keys:\n", outputs.keys(),end='\n\n')
    #print("Sequence Logits:\n",outputs,end='\n\n')
    predictions = torch.topk(outputs.logits,4)
    #print('Preditions:\n',predictions,end='\n\n')
    #print(predictions)
    torch.cuda.empty_cache()
    return predictions

In [ ]:
single_test("ok do they delver food")[1][0]

tensor([  0,   6, 101,   1])

In [ ]:
print(cluster[36])

{'hotel_110094_16', 'hotel_110033_5', 'hotel_110157_16'}


In [ ]:
for x in log[0]:
    print(x['text'])

In [ ]:
!pip install transformers

In [ ]:
import json

f3 = open('/content/drive/MyDrive/Research/ner/intent_classifier/final_result/kb_eval_final.json','r')
kb = json.load(f3)
f3.close()

id_to_name_map = {}

for cat in kb:
    for elem in kb[cat]:
        doc_lst = kb[cat][elem]['docs']
        name = kb[cat][elem]['name']
        for entry in doc_lst:
            id = doc_lst[entry]['doc_id']
            if id in id_to_name_map:
                print('What')
            else:
                id_to_name_map[id] = name

print(id_to_name_map['hotel_0_1'])

A AND B GUEST HOUSE


In [ ]:
"Alldal's;".upper()

"ALLDAL'S;"

In [ ]:
#%%writefile Baidu_eval_generation_wTemplate_IntentModel_v2.py
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig
import string
import json
import pickle
from tqdm import tqdm


def main():  
    def _get_title_body_pair(kb, doc_id):
        #gets title_ex and body_ex
        cat,id,d_id = doc_id.split('_')
        body_ex = kb[cat][id]['docs'][d_id]['body_ex']
        title_ex = kb[cat][id]['docs'][d_id]['title_ex']
        return (title_ex, body_ex)
    
    def __convert_to_tgt(domain, entity, pair):
        field_names = ["[DOMAIN]", "[ENTITY]", "[TITLE]", "[BODY]"]
        field_values = [domain]
        if entity is not None:
            field_values.append(entity)
        else:
            field_values.append("*")
        field_values.append(" ".join(pair[0].split()))
        field_values.append(" ".join(pair[1].split()))
    
        fields = [name + " " + value for name, value in zip(field_names, field_values)]
        return " ".join(fields)
    
    def single_test(title,k):
        result = tokenizer(title, return_tensors="pt")
        result.to(device)
        model.eval()
        outputs = model(**result)
        predictions = torch.topk(outputs.logits,k)
        torch.cuda.empty_cache()
        return predictions

    f1 = open('/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_logs.json','r')
    log = json.load(f1)
    f1.close()

    f2 = open('predict.json','r')
    label = json.load(f2)
    f2.close()

    f3 = open('/content/drive/MyDrive/Research/ner/intent_classifier/final_result/kb_eval_final.json','r')
    kb = json.load(f3)
    f3.close()
    
    f4 = open('/content/drive/MyDrive/Research/ner/text_classifier/final_results/finale.pkl','rb')
    cluster = pickle.load(f4)
    f4.close()

    id_to_name_map = {}

    for cat in kb:
        for elem in kb[cat]:
            doc_lst = kb[cat][elem]['docs']
            name = kb[cat][elem]['name']
            for entry in doc_lst:
                id = doc_lst[entry]['doc_id']
                if id in id_to_name_map:
                    print('What')
                else:
                    id_to_name_map[id] = name

    model_name = "bert-base-uncased"
    batch_size = 32
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    config = AutoConfig.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, padding = True, truncation = True)
    model = AutoModelForSequenceClassification.from_pretrained("fdstc10_intent_MT1_3")
    
    # Set device
    model.to(device)
    
    n = 0
    out = open('prediction_set_x.txt','w')
    out.write("src\ttgt\n")
    
    for i in range(len(log)):
        if label[i]['target'] == 1:
            dialog = []
            entity_lst = label[i]['ne_rank']
            for turn in log[i]:
                dialog.append(f"{turn['text']}\x01{1 if turn['speaker'] == 'U' else 0}")
            dialog = " [SEP] ".join(dialog)
            cluster_ids = single_test(turn['text'],3)[1][0]
            if 0 in cluster_ids:
                cluster_ids = single_test(turn['text'],4)[1][0]
            for cluster_id in cluster_ids:
                if cluster_id == 0:
                    continue
                for doc_id in cluster[int(cluster_id)]:
                    entity_name = id_to_name_map[doc_id]
                    for entry in entity_lst:
                        entity = entry['entity_name']
                        if entity == entity_name:
                            n += 1
                            domain = doc_id.split('_')[0]
                            pair = _get_title_body_pair(kb, doc_id)
                            tgt = __convert_to_tgt(domain, entity, pair)
                            out.write(f"{dialog}\t{tgt}\1{0}\n")
                            print(entity, entity_name, sep='\t==\t')
    out.close()
    print(n)

if __name__ == "__main__":
    main()

#IV. With Entity Rank file

total 17408
-rw------- 1 root root      801 Sep  9 01:45 DO.train
-rw------- 1 root root   448996 Sep  9 01:46 prediction_set_1.txt
-rw------- 1 root root   297753 Sep  9 21:21 predict.json
drwx------ 2 root root     4096 Sep  9 21:36 fdstc10_intent_MT1_3
-rw------- 1 root root    76794 Sep  9 21:36 updated_DSTC10_clean_val_labels.json
-rw------- 1 root root   328693 Sep  9 21:36 finale.pkl
-rw------- 1 root root     3226 Sep  9 21:36 Baidu_eval_generation_wTemplate_IntentModel.py
-rw------- 1 root root  3542559 Sep  9 21:39 DSTC10_clean_val_logs.json
-rw------- 1 root root 13121130 Sep  9 21:39 kb_eval_final.json
